In [ ]:
import sys
import os
import logging
import pandas as pd
import datasets
from pprint import pprint
KEY = '2-NOTEBOOK'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_AIDATA': f'_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'code/external',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

In [ ]:
# 2021-12-03:00:00 before 24h. 
# 2021-12-03:00:00

# df_case_good and df_patday_all

In [ ]:
DATA_SPLIT = '_Data/4-Data_Split'
path = os.path.join(DATA_SPLIT, 'WellDoc_full.parquet')
df_case_all = pd.read_parquet(path)
# df_case_all
df_case_all['hour'] = df_case_all['ObsDT'].dt.hour

idx = df_case_all['ObsDT'].dt.minute == 0
df_case_hours = df_case_all[idx].reset_index(drop = True)
# df_case_hours



In [ ]:
df_case = df_case_hours

print(df_case.shape)
idx1 = df_case['CGMInfoBf24h-Num-RecNum'] >= 289
idx2 = df_case['MEDInfoBf24h-DietLastToNow'] <= 60 * 12
idx3 = df_case['MEDInfoBf24h-DietRecNum'] >= 1
# idx3 = df_case['CGMInfoAf2to8h-Num-RecNum'] >= 12 * 6
df_case_good = df_case[idx1 & idx2 & idx3].reset_index(drop = True)

print(df_case_good.shape)

In [ ]:
df_case_good.columns

In [ ]:
print(df_case_good.shape)

In [ ]:
# we use the df_patday_all to do the early, middle, late split

df_patday_all =df_case_good[['PID', 'Date']].value_counts().sort_index().reset_index()
df_patday_all

# df_patient_info

In [ ]:
import pandas as pd
DATA_SPLIT = '_Data/4-Data_Split'
path = os.path.join(DATA_SPLIT, 'WellDoc_patient_split_info.parquet')
df_patient_info = pd.read_parquet(path)
df_patient_info



stratum_list = [
 '18-39_1_1.0',
 '18-39_1_2.0',
 '18-39_2_1.0',
 '18-39_2_2.0',

 '40-64_1_1.0',
 '40-64_1_2.0',
 '40-64_2_1.0',
 '40-64_2_2.0',

 '65+_1_1.0',
 '65+_1_2.0',
 '65+_2_1.0',
 '65+_2_2.0',
 ]


print(df_patient_info.shape)

idx = df_patient_info['stratum'].isin(stratum_list)


df_patient_selected = df_patient_info[idx].reset_index(drop = True)# [idx1 & idx2 & idx3 & idx4].reset_index(drop = True)
print(df_patient_selected.shape)
df_patient_selected


df_patient_stratum = df_patient_selected[['PID', 'split', 'stratum', 'middle_first_date','late_first_date']]
# df_with_stratum = pd.merge(results['df'], patient_stratum, on='PID', how='left')

df_patient_stratum = df_patient_stratum.reset_index(drop = True)
df_patient_stratum['middle_first_date'] = pd.to_datetime(df_patient_stratum['middle_first_date'])
df_patient_stratum['late_first_date'] = pd.to_datetime(df_patient_stratum['late_first_date'])
# df_patday_all 
df_patient_stratum

In [ ]:
df_patday_all =df_case_good[['PID', 'Date']].value_counts().sort_index().reset_index()

# idx = df_patday_all['count'] >= 24
# print(df_patday_all.shape)
# df_patday_good = df_patday_all[idx].reset_index(drop = True)

df_patday_good = df_patday_all
print(df_patday_good.shape)
df_patday_good = pd.merge(df_patday_good, df_patient_stratum, on='PID')
print(df_patday_good.shape)
df_patday_good['Date'] = pd.to_datetime(df_patday_good['Date'])
df_patday_good

In [ ]:
def get_early_middle_late_label(row):

    if row['Date'] >= row['middle_first_date'] and row['Date'] < row['late_first_date']:
        return 'middle'
    elif row['Date'] >= row['late_first_date']:
        return 'late'
    else:
        return 'early'

df_patday_good['time_bin'] = df_patday_good.apply(get_early_middle_late_label, axis=1)
df_patday_good['time_bin'].value_counts().sort_index()
# df_patday_good['label'].value_counts().sort_index()

df_with_stratum = df_patday_good # df_patday_with_stratum
df_with_stratum


In [ ]:
stratum_list = df_with_stratum['stratum'].unique()
stratum_list

In [ ]:
stratum_list = df_with_stratum['stratum'].unique()

stratum_stats = df_with_stratum.groupby('stratum').agg({
    'PID': ['nunique', 'count']
}).reset_index()
stratum_stats.columns = ['stratum', 'n_patients', 'n_days']

stratum_stats

In [ ]:
# df_with_stratum

In [ ]:
# df_patient_selected[['stratum', 'split']].value_counts().sort_index().reset_index()

In [ ]:
stratum_list = df_patient_selected['stratum'].unique()
print(stratum_list)

In [ ]:
df_with_stratum

In [ ]:
final_df = df_with_stratum
final_df['split_timebin'] = final_df['split'] + '-' + final_df['time_bin']
final_df[['split_timebin']].value_counts().sort_index().reset_index()

In [ ]:
df_case_good.shape

In [ ]:
df_case_good.head()

In [ ]:
df_case_good['Date'] = pd.to_datetime(df_case_good['Date'])

df_case_pretrain_split = pd.merge(df_case_good, final_df, on = ['PID', 'Date'])

len(df_case_pretrain_split) / len(final_df)

In [ ]:
df_case_pretrain_split

In [ ]:
path = os.path.join(DATA_SPLIT, 'WellDoc_ds_case_dietevent_split.parquet')
df_case_pretrain_split.to_parquet(path)

In [ ]:
df_count = df_case_pretrain_split[['stratum', 'split_timebin']].value_counts().sort_index().reset_index()
df_table = df_count.pivot(index = 'stratum', columns = 'split_timebin', values = 'count').fillna(0)

cols = [ 'train-early', 'train-middle', 'train-late', 
        'val-early', 'val-middle', 'val-late', 
        'test-early', 'test-middle', 'test-late'
        ]

df_table = df_table[cols]

# Add row totals
df_table['Total'] = df_table.sum(axis=1)

# Add column totals
df_table.loc['Total'] = df_table.sum()

df_table = df_table[df_table['train-early'] > 0].reset_index()
df_table

In [ ]:
# df_case_pretrain_split

In [ ]:
DATA_SPLIT = '_Data/4-Data_Split'


path = path # os.path.join(DATA_SPLIT, 'WellDoc_ds_case_pretrain_split.parquet')
# df_case_pretrain_split.to_parquet(path)

df_case_pretrain_split = pd.read_parquet(path)
df_case_pretrain_split